In [ ]:
import numpy as np
import MDAnalysis as mda
import nmrformd

In [ ]:
import sys, os, git
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

In [ ]:
def save_result(data, name = "intra_H2O"):
    """Save the correlation functions in dictionary"""
    if not os.path.exists("raw_data/"):
        os.makedirs("raw_data/")
    saving_file = "raw_data/" + name + ".npy"
    t = data.t
    f = data.f
    C = data.gij[0]
    R1 = data.R1
    R2 = data.R2
    N = data.group_j.atoms.n_atoms
    try:
        previous_dictionary = np.load(saving_file, allow_pickle=True)
        t_prev = np.real(previous_dictionary.item()["t"])
        assert len(t_prev) == len(t)
        C_prev = np.real(previous_dictionary.item()["C"])
        R1_prev = np.real(previous_dictionary.item()["R1"])
        R2_prev = np.real(previous_dictionary.item()["R2"])
        N_prev = np.real(previous_dictionary.item()["N"])
        C = (C*N + C_prev*N_prev) / (N_prev + N)
        R1 = (R1*N + R1_prev*N_prev) / (N_prev + N)
        R2 = (R2*N + R2_prev*N_prev) / (N_prev + N)
        N += N_prev
    except:
        pass
    dictionary = {
    "t": t,
    "f": f,
    "C": C,
    "N": N,
    "R1": R1,
    "R2": R2,
    }
    np.save(saving_file, dictionary)
    return N

In [ ]:
while 1<2:
    for T in ["0.6", "0.8", "1.0", "1.2", "1.4", "1.6", "1.8", "2.0", "2.2", "2.4", "2.6", "2.8", "3.0", "3.2"]:
        datapath = git_path + "/nmrformd-data/lennard-jones-fluid/raw-data/T"+T+"/"
        u = mda.Universe(datapath+"system.data", datapath+"dump.xtc")
        ALL = u.select_atoms("all")
        nmr = nmrformd.NMR(u, atom_group = ALL, number_i=1)
        N = save_result(nmr, name = "T"+T)
        if T == "0.6":
            print("Number of cycle:")